<a href="https://colab.research.google.com/github/nh7881/machine_learning_project_class/blob/master/exercise_10/assignment_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>